## COMCATE code enforcement violation summary  
Accepts standard report of annual code violations, by violation event number keyed by apn ('acctid').  This report may have multiple voilations per event, and this notebook splits those out to be counted.

Scoring the violations will require adding a table mapping violation type to scoring weight.  The current verions just counts all the violations up, as equal.

The bottom of this sheet clusters the violations by point of contact, and writes that out to a file for later use in group calculations.


In [1]:
!pip install leven
!pip install simpledbf
!pip install ngram

     |████████████████████████████████| 154 kB 5.0 MB/s 
  Created wheel for leven: filename=leven-1.0.4-cp37-cp37m-linux_x86_64.whl size=55508 sha256=029354cdc88a40eeb668af5324b9a28d2a2c4904e223b24a2c9ed09b196b8ddf
  Stored in directory: /root/.cache/pip/wheels/27/d5/b1/b36dd23f85aa662f19b11ef6069d5dc66643dfc1831d9f2bf5
Successfully built leven
  Created wheel for simpledbf: filename=simpledbf-0.2.6-py3-none-any.whl size=13801 sha256=b389a1ed22972372686e973c28b7bc2077465a1afb53d2259cb2d6da7d2b3b8c
  Stored in directory: /root/.cache/pip/wheels/24/43/f4/39ad84349e5358346be977fe626160f5625fdd3ea8e017518c
Successfully built simpledbf
  Created wheel for ngram: filename=ngram-3.3.2-py3-none-any.whl size=24740 sha256=224cc93f1330c0c58dc3d2b592890a5ee161002d7d978b56ddc024630d5b2d4c
  Stored in directory: /root/.cache/pip/wheels/24/09/ee/4d02ab106d48aa517b1295739e8f9c56ba159a3b61b43a64cf
Successfully built ngram


In [2]:
import pandas as pd
import re
from simpledbf import Dbf5
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Load the sdat property dataset

This data is used when trying to fix the code violation to add missing tax ids

In [128]:
sdat_columns = ['acctid',
       'ownname1','ownname2', 'namekey', 'ownadd1', 'ownadd2', 
       'owncity', 'ownstate', 'ownerzip', 'ownzip2']
rental_columns = ['Business Name', 'License Id', 'address',
       'Customer Id', 'Dist/Account No', 'RENTAL']
rental_merge_columns= sdat_columns + rental_columns
violation_merge_columns = ['address']#,'Open_Date','Follow Up Date']

### Enrich the billing data with columns from prior year(s), to get started

In [129]:
cambridge_apns = pd.read_csv('/content/drive/My Drive/SDAT/CAN-ref.csv')
cambridge_apns = cambridge_apns[~cambridge_apns["acctid"].str.contains("-")]
cambridge_apns.acctid = cambridge_apns.acctid.astype(int)
cambridge_apns

,acctid,CAN_GROUP,CAN_OWNCLASS,GEOLATLON,GEOHASH
0,1007155603,NaN,COMMERCIAL,"38.55839822712713, -76.07416153313052",dqcgkgk6zw3u
1,1007161174,NaN,COMMERCIAL,"38.572730034552904, -76.07313861641092",dqcgkvvnrqh9
2,1007117892,NaN,COMMERCIAL,"38.55833133614624, -76.07311598366888",dqcgkgm4xcvm
3,1007132581,NaN,HOUSING,"38.56835512643795, -76.06357647998182",dqcgmjhhynqx
4,1007122225,NaN,HOUSING,"38.570440067232006, -76.07000091981125",dqcgkvx90e0j
...,...,...,...,...,...
7919,1007136684,CHITLIK,HOUSING,"38.572767623844605, -76.07923081301398",dqcgkvby9mjc
7920,1007139101,MAIN_ST,OTHER,"38.56571795272529, -76.0666919812559",dqcgmh9vtdgk
7921,1007139411,MAIN_ST,OTHER,"38.57344340088598, -76.08106406712523",dqcgkwp6wenb
7922,1007137281,MAIN_ST,COMMERCIAL,"38.5689638888623, -76.07786016063024",dqcgkv3b3rgz


In [130]:
sdat = pd.read_csv('drive/My Drive/pita 2021/SDAT-CAN-ref-202105.csv')
sdat.acctid = sdat.acctid.astype(int)
sdat.loc[sdat.ownname1.str.contains("CAMBRIDGE HOUSING AUTHORITY"), 'ownname2'] = "JAMES B. CHANEY, JR"
sdat.loc[sdat.ownname1.str.contains("CAMBRIDGE HOUSING AUTHORITY"), 'ownadd1'] = "5665 FEDERALSBURG HWY"


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,11,12,13,14,15,16,29,38,45,49,52,53,77,82,86,104,106,112,113,129,137) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## MOVE THIS TO RENTAL REGISTRATION ETL?

In [131]:
def format_property_location(x):
  addr = re.sub(r"(^.*)( [ \d]+\w?)$", r"\1", x.strip().upper())
  addr = re.sub(" DOWN.*$",'',addr).strip()
  return addr

rentals = pd.read_csv('/content/drive/My Drive/pita 2021/cleaned_rental_billing-2021.csv')
rentals['address'] = rentals['Property Location'].apply(lambda x: format_property_location(x))
rentals.address = rentals.apply(lambda x: re.sub(r' {2,}', ' ', str(x.address)),axis=1)
print("base:",len(rentals))
rentals.drop_duplicates(subset=['acctid','Customer Id','address'],keep='last',inplace=True)

print("removing mutiple licenses per unique acctid-address:",len(rentals))
rentals = rentals.merge(sdat[sdat_columns],on='acctid',how='outer',indicator=True)
print(len(rentals.query('_merge == "both"')))
temp = rentals.query('_merge == "left_only"')[rental_columns].merge(sdat,on='address',how='outer',indicator=True)
rentals = pd.concat([rentals.query('_merge =="both"')[rental_merge_columns],temp.query('_merge == "both" and city == "CAMBRIDGE"')[rental_merge_columns]])
print(len(rentals))
temp.query('_merge == "left_only"')

base: 1462
removing mutiple licenses per unique acctid-address: 1462
1456
1460


,Business Name,License Id,address,Customer Id,Dist/Account No,RENTAL,acctid,jurscode,digxcord,digycord,ct2010,bg2010,geogcode,ooi,resityp,strtnum,strtdir,strtnam,strttyp,strtsfx,strtunt,addrtyp,city,zipcode,ownname1,ownname2,namekey,ownadd1,ownadd2,owncity,ownstate,ownerzip,ownzip2,premsnum,premsdir,premsnam,premstyp,premcity,premzip,premzip2,...,fcmacode,agfndarea,agfndluom,entzndat,entznassm,plndevdat,nprctstdat,nprcarea,nprcluom,homqlcod,homqldat,bldg_story,bldg_units,resident,resi2010,resi2000,resi1990,resiuths,aprtment,trailer,special,other,ptype,sdatwebadr,existing,mdpvdate,sdat,google_maps,struct_sqft,assessed_value,address_number,address_unit_id,street_direction,street_name,street_type,premise_address_type_mdp_field_premstyp_sdat_field_24,premise_address_city_mdp_field_premcity_sdat_field_25,premise_address_zip_code_mdp_field_premzip_sdat_field_26,mdp_street_address_mdp_field_address,_merge
0,JAMES DIXON,20-00116,738 BAYLY AVE,RR-01094,07-069094,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,HAMBLETON CREEK PROP LLC,20-00508,501 ACADEMY ST APT,RR-03828,07-146310,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [132]:
rental_aggregation_map = dict(zip(rentals.columns,['first'] * len(rentals.columns)))
rental_aggregation_map['address'] = ','.join
rental_aggregation_map['License Id'] = ','.join
rental_aggregation_map['RENTAL'] = 'sum'
rentals = rentals.groupby('acctid',as_index=False).agg(rental_aggregation_map)
rentals.query('address.str.contains("WOODS")',engine='python')

,acctid,ownname1,ownname2,namekey,ownadd1,ownadd2,owncity,ownstate,ownerzip,ownzip2,Business Name,License Id,address,Customer Id,Dist/Account No,RENTAL
908,1.007160e+09,CAMBRIDGE CLUB ASSOCIATES LIMITED,PARTNERSHIP,CAMBRIDGE CLUB ASSOCIATES,STE 200,726 YORKLYN RD,HOCKESSIN,DE,19707.0,8701.0,CAMBRIDGE CLUB ASSOC LP,"20-01067,20-01068,20-01069,20-01070,20-01071,2...","5216 WOODS ROAD BLDG,5218 WOODS ROAD BLDG,5220...",RR-09003,07-159641,76.0


#### Format violations list dates and violations lists
pandas df.explode() is used to make records for each row with seprate violations, but we won't do that until needed to avoid memeory pressure when clustering.

In [133]:
violations = pd.read_csv("drive/My Drive/comcate/code_violations-20210701.csv")
violations['Open Date'] = violations.apply(lambda x: pd.to_datetime(x['Open Date']), axis = 1)
violations = violations.rename(columns={'Open Date':'Open_Date'})

In [134]:
import re
print(len(violations))
violations.address = violations.apply(lambda x: re.sub(r' {2,}', ' ', str(x.address)),axis=1)

3673


In [135]:
violations["Follow Up Date"] = pd.to_datetime(violations["Follow Up Date"])

def get_aggregation_map(df,prefix='type_'):
  score_cols = {x:'sum' for x in df.columns if prefix in x}
  not_score_cols = {x:'first' for x in df.columns if prefix not in x}
  not_score_cols['Open_Date'] = 'min'
  not_score_cols['Follow Up Date'] = 'max'
  return {**score_cols, **not_score_cols}

#### fixup account ids

In [136]:
violations.drop_duplicates(inplace=True)
violations.APN = violations.APN.fillna(0).astype(int)
violations.APN = violations.APN.astype(str)
violations = violations.query('APN.str.contains("[0-9]+")',engine='python')

In [137]:
# the order of the columns in the spreadsheet allows merging of results to 'data studio'
ordered_columns = [
 'acctid','type_0_sum','type_1_sum','type_2_sum','type_3_sum','type_4_sum','property_condition_score','property_condition_tier',
 'ownname1','ownname2','namekey','ownadd1','ownadd2','owncity','ownstate','ownerzip','ownzip2',
 'Business Name','address','Customer Id','Dist/Account No','RENTAL',\
 'CAN_GROUP','CAN_OWNCLASS','GEOLATLON','GEOHASH',
 'fee','std_fee',
 'Open_Date','Follow_Up_Date']


In [138]:
import re

def split_violations(x):
  temp = re.sub(r'[-\n:]+','',x).lower().strip()
  temp = re.split(r'(closed|open)',temp)
  return [x.strip() for x in temp if not 'closed' in x and not 'open' in x and x.strip() != '']

print(len(violations))
violations['vtypes'] = violations.apply(lambda x: split_violations(x['Violation(s)']),axis=1)

v_flat = violations.explode('vtypes')
len(v_flat)

3673


6502

In [139]:
v_categories = pd.read_csv('/content/drive/My Drive/pita 2021/violation_types.csv')
v_categories.violation_types = v_categories.violation_types.apply(lambda x: x.lower())
v_lookup = dict(zip(v_categories.violation_types,v_categories.category))
v_flat['violation_category_text'] = v_flat.vtypes.apply(lambda x: x.split('-')[0].strip())

def lookup_category(x):
  if x in v_lookup.keys():
    return v_lookup[x]
  else:
    return 0

v_flat['type_0'] = v_flat.apply(lambda x: 1 if lookup_category(x.violation_category_text) < 1 else 0,axis=1)
v_flat['type_1'] = v_flat.apply(lambda x: 1 if lookup_category(x.violation_category_text) == 1 else 0,axis=1)
v_flat['type_2'] = v_flat.apply(lambda x: 1 if lookup_category(x.violation_category_text) == 2 else 0,axis=1)
v_flat['type_3'] = v_flat.apply(lambda x: 1 if lookup_category(x.violation_category_text) == 3 else 0,axis=1)
v_flat['type_4'] = v_flat.apply(lambda x: 1 if lookup_category(x.violation_category_text) == 4 else 0,axis=1)

#### Summarize violations of all types.
Split the multi-line violation descriptions, and total the number of individual issues per tax account id.

In [140]:
v_flat.drop(columns=[x for x in v_flat.columns if 'Unnamed' in x])
v_flat.drop(columns=['vtypes'])

,Unnamed: 0,Case,eFM Case,Module,Status,Violation(s),Priority,Street Number,Street Direction,Street Name,Street Type,Unit,City,APN,Initiation,Open_Date,Follow Up Date,Contacts,Assigned To,address,violation_category_text,type_0,type_1,type_2,type_3,type_4
0,0,2019-926,NaN,Code Enforcement,Closed: Voluntary Compliance,Accumulation of Rubbish/Garbage - Closed,Medium,507,NaN,DUNNS,CT,NaN,CAMBRIDGE,1007157169,Neighborhood Patrol,2019-10-23,2019-10-30,Property Owner MANOKEY CALVIN L507 DUNNS Ct CA...,"*Jackson, Nancy",507 DUNNS CT,accumulation of rubbish/garbage,0,0,0,0,1
1,1,2019-749,NaN,Code Enforcement,Closed: Voluntary Compliance,Sec 4-42 Registration of non owner occupied re...,Medium,202,NaN,MARKET,SQ,NaN,Cambridge,1007193793,MUC,2019-09-18,2019-10-18,Property OwnerDUANE MARSHALL LYNN MARSHALL5004...,"Webb,, Susan",202 MARKET SQ,sec 442 registration of non owner occupied res...,1,0,0,0,0
2,2,2019-731,NaN,Code Enforcement,Closed: Voluntary Compliance,Weeds/Sidewalks - Closed,Medium,411,NaN,PINE,ST,NaN,CAMBRIDGE,1007105991,Neighborhood Patrol,2019-09-12,2019-09-19,Property Owner US LIENS 401K TRUST PO BOX 352 ...,"*Jackson, Nancy",411 PINE ST,weeds/sidewalks,0,0,0,0,1
3,3,2019-581,NaN,Code Enforcement,Closed: City Abated,Weeds/Sidewalks - Closed,Medium,601,NaN,GLASGOW,ST,NaN,CAMBRIDGE,1007146752,Neighborhood Patrol,2019-08-15,2019-08-22,Property OwnerKENNETH E. & CONSTANCE R FISHER3...,"*Jackson, Nancy",601 GLASGOW ST,weeds/sidewalks,0,0,0,0,1
4,4,2019-571,NaN,Code Enforcement,Closed: Voluntary Compliance,Trees or Shrubbery Overhanging Street or Sidew...,Medium,406,NaN,MUIR,ST,NaN,CAMBRIDGE,1007158440,Neighborhood Patrol,2019-08-14,2019-08-21,Property Owner THOMAS B ELIASON406 ACADEMY St ...,"*Jackson, Nancy",406 MUIR ST,trees or shrubbery overhanging street or sidewalk,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3670,4,2020-646,NaN,Code Enforcement,Closed: Unfounded,301.3 Vacant Structures and Land - ClosedWeeds...,Medium,NaN,NaN,NAN,NAN,NaN,NaN,1007142382,Complaint,2020-05-14,2020-05-21,Property Owner Slaughter Harold Thedore3055 Ha...,"Parks, Todd",KENT ST,301.3 vacant structures and land,1,0,0,0,0
3670,4,2020-646,NaN,Code Enforcement,Closed: Unfounded,301.3 Vacant Structures and Land - ClosedWeeds...,Medium,NaN,NaN,NAN,NAN,NaN,NaN,1007142382,Complaint,2020-05-14,2020-05-21,Property Owner Slaughter Harold Thedore3055 Ha...,"Parks, Todd",KENT ST,weeds/sidewalks,0,0,0,0,1
3670,4,2020-646,NaN,Code Enforcement,Closed: Unfounded,301.3 Vacant Structures and Land - ClosedWeeds...,Medium,NaN,NaN,NAN,NAN,NaN,NaN,1007142382,Complaint,2020-05-14,2020-05-21,Property Owner Slaughter Harold Thedore3055 Ha...,"Parks, Todd",KENT ST,accumulation of rubbish/garbage,0,0,0,0,1
3671,5,2021-162,NaN,Code Enforcement,Closed - Error: Written in Error,Weeds/Sidewalks - Closed,Medium,NaN,NaN,NAN,NAN,NaN,NaN,1007154755,Neighborhood Patrol,2021-05-05,2021-05-14,Property Owner Madison Bay Holdings LLC1240 Pe...,"Seeberger, Larry",221 HENRY ST,weeds/sidewalks,0,0,0,0,1


In [141]:
import numpy as np
ht_score = v_flat.groupby(['APN','address']).agg({'Open_Date':'min','Follow Up Date':'max','type_0':['sum'],'type_1':['sum'],'type_2':['sum'],'type_3':['sum'],'type_4':['sum']})
#ht_score = v_flat.groupby(['APN','address']).agg(scoring_map)
ht_score.columns = ['_'.join(col).strip() for col in ht_score.columns.values]
ht_score

,,Open_Date_min,Follow Up Date_max,type_0_sum,type_1_sum,type_2_sum,type_3_sum,type_4_sum
APN,address,,,,,,,
0,860 GREENWOOD AVE,2018-09-04,2018-09-11,0,0,0,0,1
1007104014,405 E APPLEBY AVE,2020-05-28,2020-06-04,0,0,0,0,1
1007104057,304 DORCHESTER AVE,2019-11-12,2019-11-19,0,0,1,0,2
1007104065,413 SHEPHERD AVE,2020-04-01,2020-09-03,0,0,1,2,3
1007104138,110 CHOPTANK AVE,2018-09-25,2019-05-20,0,0,0,0,2
...,...,...,...,...,...,...,...,...
1007229348,1231 ZACHARY DR,2018-10-01,2018-10-17,0,0,0,1,0
1007230664,608 WATER ST,2020-08-25,2020-09-08,0,0,0,0,2
1007230680,703 BAYLY ROAD,2019-09-10,2019-09-17,0,0,0,0,1


In [142]:
housing_quality_tier_scores = [0,6,5,1,0.5,10]

lo_bldg_by_tier = {1:80,2:85,3:100}
hi_bldg_by_tier = {1:140,2:165,3:195}
lo_unit_by_tier = {1:30,2:60,3:160}
hi_unit_by_tier = {1:10,2:20,3:80}
building_fees_by_capacity = {'low': lo_bldg_by_tier,'high': hi_bldg_by_tier}
unit_fees_by_capacity = {'low': lo_unit_by_tier ,'high': hi_unit_by_tier}
supplemental_fee_by_tier = {'low':{1:0,2:100,3:205},'high':{1:0,2:155,3:410}}

def pc_score(x): 
  return x.type_1_sum * housing_quality_tier_scores[1] + \
  x.type_2_sum * housing_quality_tier_scores[2] + \
  x.type_3_sum * housing_quality_tier_scores[3] + \
  x.type_4_sum * housing_quality_tier_scores[4]

def pc_tier(x):
  if x > 39:
    return 3
  elif x > 19:
    return 2
  else:
    return 1

def pc_fee(tier,units):
  fee = 0
  per_unit_fee = 0
  building_fee = 0
  capacity_fee = 0
  sup_fee = 0

  if units > 3:
    building_fee = building_fees_by_capacity['high'][tier]
    per_unit_fee = unit_fees_by_capacity['high'][tier]
    capacity_fee = per_unit_fee * units
    sup_fee = supplemental_fee_by_tier['high'][tier]
  else:
    building_fee = building_fees_by_capacity['low'][tier]
    per_unit_fee = unit_fees_by_capacity['low'][tier]
    capacity_fee = per_unit_fee * units
    sup_fee = supplemental_fee_by_tier['low'][tier]
  fee = building_fee + capacity_fee + sup_fee
  if tier  > 1:
    print(tier,units,per_unit_fee,building_fee,'+',capacity_fee,'+',sup_fee,'=',fee,'vs',units*56)
  return fee

ht_score['property_condition_score'] = ht_score.apply(lambda x: pc_score(x),axis=1)
#ht_score['property_condition_tier'] = ht_score.apply(lambda x: pc_tier(x.property_condition_score),axis=1)
ht_score = ht_score.reset_index()
ht_score.rename(columns={'APN':'acctid'},inplace=True)
ht_score.acctid = ht_score.acctid.astype(int)

ht_score[ht_score.property_condition_score > 40]

,acctid,address,Open_Date_min,Follow Up Date_max,type_0_sum,type_1_sum,type_2_sum,type_3_sum,type_4_sum,property_condition_score
18,1007104863,405 HENRY ST,2019-01-16,2021-05-13,2,2,5,11,11,53.5
72,1007107730,832 WASHINGTON ST,2018-08-30,2021-05-14,2,4,5,9,16,66.0
154,1007113358,813 RACE ST,2019-06-18,2021-05-17,2,7,5,10,11,82.5
191,1007117345,802 HIGH ST,2018-06-04,2021-05-13,0,5,3,6,10,56.0
282,1007122047,802 WASHINGTON ST,2019-10-01,2021-01-04,0,0,8,0,16,48.0
291,1007122195,1016 PINE ST,2018-06-08,2020-05-08,0,6,0,1,11,42.5
315,1007123655,521 PINE ST,2018-07-19,2020-02-05,1,5,2,6,5,48.5
371,1007126034,909 MACES LANE,2018-07-03,2020-09-10,1,2,5,7,18,53.0
374,1007126166,605 DOUGLAS ST,2019-01-25,2020-09-01,2,5,6,16,2,77.0
384,1007127049,509 EDGEWOOD AVE,2019-03-01,2021-06-21,2,10,16,25,17,173.5


In [143]:
results = ht_score.drop(columns='address').merge(rentals[['acctid','RENTAL','License Id','Business Name', 'Customer Id', 'Dist/Account No']],on='acctid',how='right')
results.columns

aggregation_map = dict(zip(results.columns,['first' for k in range(len(results.columns))]))
for k in ['type_0_sum','type_1_sum', 'type_2_sum', 'type_3_sum','type_4_sum','property_condition_score']:
  aggregation_map[k] = 'sum'

In [144]:
#results = ht_score.drop(columns='address').merge(rentals,on='acctid',how='right')
#results.property_condition_tier = results.property_condition_tier.fillna(1)
results = results.fillna(0)
print(len(results))
results = results.merge(cambridge_apns,on='acctid',how='inner')

# THE FIX!!
#results = results.groupby(['acctid','License Id']).agg(aggregation_map)
results = results.groupby(['acctid']).agg(aggregation_map)
results['property_condition_tier'] = results.apply(lambda x: pc_tier(x.property_condition_score),axis=1)
results = results.drop(columns=['acctid','License Id']).reset_index()


1457


### CALCULATE THE FEES

In [145]:
results['fee'] = results.apply(lambda x: pc_fee(x.property_condition_tier,x['RENTAL']),axis=1)
results['std_fee'] = results.apply(lambda x: x['RENTAL'] * 56,axis=1)
results.accitid = results.acctid.astype(int)
results.drop_duplicates(inplace=True)
#results[results.property_condition_tier > 2]


2 2.0 60 85 + 120.0 + 100 = 305.0 vs 112.0
2 1.0 60 85 + 60.0 + 100 = 245.0 vs 56.0
2 1.0 60 85 + 60.0 + 100 = 245.0 vs 56.0
2 4.0 20 165 + 80.0 + 155 = 400.0 vs 224.0
2 2.0 60 85 + 120.0 + 100 = 305.0 vs 112.0
3 1.0 160 100 + 160.0 + 205 = 465.0 vs 56.0
2 1.0 60 85 + 60.0 + 100 = 245.0 vs 56.0
2 1.0 60 85 + 60.0 + 100 = 245.0 vs 56.0
3 1.0 160 100 + 160.0 + 205 = 465.0 vs 56.0
3 4.0 80 195 + 320.0 + 410 = 925.0 vs 224.0
3 1.0 160 100 + 160.0 + 205 = 465.0 vs 56.0
3 1.0 160 100 + 160.0 + 205 = 465.0 vs 56.0
2 1.0 60 85 + 60.0 + 100 = 245.0 vs 56.0
2 2.0 60 85 + 120.0 + 100 = 305.0 vs 112.0
3 2.0 160 100 + 320.0 + 205 = 625.0 vs 112.0
2 2.0 60 85 + 120.0 + 100 = 305.0 vs 112.0
3 2.0 160 100 + 320.0 + 205 = 625.0 vs 112.0
3 121.0 80 195 + 9680.0 + 410 = 10285.0 vs 6776.0
2 1.0 60 85 + 60.0 + 100 = 245.0 vs 56.0
2 1.0 60 85 + 60.0 + 100 = 245.0 vs 56.0
2 1.0 60 85 + 60.0 + 100 = 245.0 vs 56.0
2 1.0 60 85 + 60.0 + 100 = 245.0 vs 56.0
2 1.0 60 85 + 60.0 + 100 = 245.0 vs 56.0
2 1.0 60 85 + 6

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


In [146]:
sdat.strtnum = sdat.strtnum.fillna(0).astype(int).astype(str)
sdat.strtdir = sdat.strtdir.fillna("")
sdat.strtnam = sdat.strtnam.fillna("")
sdat.strttyp = sdat.strttyp.fillna("")
sdat.strtunt = sdat.strttyp.fillna("")
sdat.premsnum = sdat.premsnum.fillna(0).astype(int).astype(str)
sdat.premsdir = sdat.premsdir.fillna("")
sdat.premsnam = sdat.premsnam.fillna("")
sdat.premstyp = sdat.premstyp.fillna("")
sdat.address = sdat.address.astype(str).fillna("").replace('nan','')

def get_best_sdat_address(id,debug=False):
  base_address = rentals[rentals.acctid == id].address.item().strip()
  if debug:
    print(id,base_address)
  x = sdat[sdat.acctid == id]
  strtnum = x.strtnum.item() if x.strtnum.item() != "0" else ""
  premsnum = x.premsnum.item() if x.premsnum.item() != "0" else ""
  address = x.address.item().strip()
  alt_address = ""

  # prefer unit number available
  strtdir = x.strtdir.item().strip()
  strtnam = x.strtnam.item().strip()
  strttyp = x.strttyp.item().strip()
  strtunt = x.strtunt.item().strip() if not x.strtunt.item().strip() == strttyp else ""
  alt_address = re.sub(r" +"," "," ".join([strtnum,str(strtdir),str(strtnam),str(strttyp),str(strtunt)])).strip().replace('nan','').replace(r' +',' ')
  # prefer sdat address of some sort, so see if 'premises' value is available
  if len(alt_address) == 0:
    strtdir = x.premsdir.item().strip()
    strtnam = x.premsnam.item().strip()
    strttyp = x.premstyp.item().strip()
    strtunt = ""
    alt_address = re.sub(r" +"," "," ".join([str(premsnum),str(strtdir),str(strtnam),str(strttyp),str(strtunt)])).strip().replace('nan','').replace(r' +',' ')

  estimated_address = ""
  # use the value from comcate if nothing else is available
  if (len(address) == 0 or address.isspace()) or (len(alt_address) or alt_address.isspace()):
    estimated_address = sdat[sdat.acctid == id].legal2.item()#.fillna("").strip()
  else:
    estimated_address = address if len(address) >= len(alt_address) else alt_address

  return estimated_address if len(base_address) == 0 or len(base_address) < len(estimated_address) else base_address

get_best_sdat_address(1007111649,True)
#sdat[sdat.acctid == 1007104057]

1007111649 705-707 ROSEMOUNT AVE


'705-707 ROSEMONT AVENUE'

In [147]:
# for use later...

#sdat.ownname1 = sdat.ownname1.apply(lambda x: re.sub('KNOX CHRISTOPHER.+','KNOX CHRISTOPHER',x))
results[results.duplicated(subset=['acctid','Business Name', 'Customer Id', 'Dist/Account No'],keep=False)]
results = results.merge(cambridge_apns,on='acctid',how='left')
results.rename(columns={'Open_Date_min':'Open_Date','Follow Up Date_max':'Follow_Up_Date'},inplace=True)
old_results = results.copy()
results = results.merge(sdat[sdat_columns + ['address']],on='acctid',how='inner')
print(len(old_results),len(rentals),len(results))
print(results.columns)
results.Follow_Up_Date = results.Follow_Up_Date.apply(lambda x: x if str(x) != "0" else pd.to_datetime('today'))
results.Open_Date = results.Open_Date.apply(lambda x: x if str(x) != "0" else pd.to_datetime('today'))

results.address = results.address.fillna("")
results.address = results.apply(lambda x: x.address if x.address != "nan" else "",axis=1)
results.address = results.apply(lambda x: get_best_sdat_address(x.acctid,False),axis=1) # False means don't print 
#results[results.ownname1.str.contains("CAMBRIDGE HOUSING AUTHORITY")]

1449 1450 1449
Index(['acctid', 'Open_Date', 'Follow_Up_Date', 'type_0_sum', 'type_1_sum',
       'type_2_sum', 'type_3_sum', 'type_4_sum', 'property_condition_score',
       'RENTAL', 'Business Name', 'Customer Id', 'Dist/Account No',
       'property_condition_tier', 'fee', 'std_fee', 'CAN_GROUP',
       'CAN_OWNCLASS', 'GEOLATLON', 'GEOHASH', 'ownname1', 'ownname2',
       'namekey', 'ownadd1', 'ownadd2', 'owncity', 'ownstate', 'ownerzip',
       'ownzip2', 'address'],
      dtype='object')


In [148]:
results[ordered_columns].to_csv('drive/My Drive/pita 2021/property_tiering-20210724.csv')


In [49]:
results#[results.acctid == 1007106432]
#sdat[sdat.acctid == 1007106432][['strtnum','strtnam']]
#results[results.acctid == 1007105339].apply(lambda t: get_best_sdat_address(t.acctid,True),axis=1)
rentals[rentals.acctid == 1007127049]#.drop_dupicates()
#rentals[rentals.duplicated(subset=['acctid'])]
# 145314  1101 GLOVER AVE
rentals.acctid = rentals.acctid.astype(int)
rentals[rentals.duplicated(subset=['acctid'],keep=False)]

,acctid,ownname1,ownname2,namekey,ownadd1,ownadd2,owncity,ownstate,ownerzip,ownzip2,Business Name,License Id,address,Customer Id,Dist/Account No,RENTAL
281,1007127049,GLG HOLDINGS LLC,NaN,GLG HOLDINGS LLC,C/O THOMAS BRADLEY,811 LOCUST ST,CAMBRIDGE,MD,21613.0,NaN,GLG HOLDINGS LLC,20-00291,509 EDGEWOOD AVE,RR-03214,07-127049,2.0
282,1007127049,GLG HOLDINGS LLC,NaN,GLG HOLDINGS LLC,C/O THOMAS BRADLEY,811 LOCUST ST,CAMBRIDGE,MD,21613.0,NaN,GLG HOLDINGS LLC,20-00292,517 EDGEWOOD AVE,RR-03214,07-127049,2.0
800,1007163495,SKINNERS COURT LLC,NaN,SKINNERS COURT LLC,3550 PATUXENT RIVER RD,NaN,DAVIDSONVILLE,MD,21035.0,2230.0,JAMES DUSWALT,20-00845,420 SKINNERS CT,RR-00519,07-163495,1.0
801,1007163495,SKINNERS COURT LLC,NaN,SKINNERS COURT LLC,3550 PATUXENT RIVER RD,NaN,DAVIDSONVILLE,MD,21035.0,2230.0,JAMES DUSWALT,20-00856,416 SKINNERS COURT,RR-00519,07-163495,2.0
830,1007145632,ARCADE LLC,NaN,ARCADE LLC,PO BOX 1118,NaN,STEVENSVILLE,MD,21666.0,1118.0,ARCADE LLC,20-00875,512-514 RACE ST,RR-07643,07-145632,8.0
831,1007145632,ARCADE LLC,NaN,ARCADE LLC,PO BOX 1118,NaN,STEVENSVILLE,MD,21666.0,1118.0,ARCADE LLC,20-00876,504 RACE ST,RR-07643,07-145632,25.0
1001,1007159641,CAMBRIDGE CLUB ASSOCIATES LIMITED,PARTNERSHIP,CAMBRIDGE CLUB ASSOCIATES,STE 200,726 YORKLYN RD,HOCKESSIN,DE,19707.0,8701.0,CAMBRIDGE CLUB ASSOC LP,20-01067,5216 WOODS ROAD BLDG,RR-09003,07-159641,16.0
1002,1007159641,CAMBRIDGE CLUB ASSOCIATES LIMITED,PARTNERSHIP,CAMBRIDGE CLUB ASSOCIATES,STE 200,726 YORKLYN RD,HOCKESSIN,DE,19707.0,8701.0,CAMBRIDGE CLUB ASSOC LP,20-01068,5218 WOODS ROAD BLDG,RR-09003,07-159641,12.0
1003,1007159641,CAMBRIDGE CLUB ASSOCIATES LIMITED,PARTNERSHIP,CAMBRIDGE CLUB ASSOCIATES,STE 200,726 YORKLYN RD,HOCKESSIN,DE,19707.0,8701.0,CAMBRIDGE CLUB ASSOC LP,20-01069,5220 WOODS ROAD BLDG,RR-09003,07-159641,12.0
1004,1007159641,CAMBRIDGE CLUB ASSOCIATES LIMITED,PARTNERSHIP,CAMBRIDGE CLUB ASSOCIATES,STE 200,726 YORKLYN RD,HOCKESSIN,DE,19707.0,8701.0,CAMBRIDGE CLUB ASSOC LP,20-01070,5222 WOODS ROAD BLDG,RR-09003,07-159641,12.0


In [ ]:
priors = pd.read_csv('drive/My Drive/pita 2021/property_tiering-20210702.csv')
priors.columns.to_list()

['Unnamed: 0',
 'acctid',
 'type_0_sum',
 'type_1_sum',
 'type_2_sum',
 'type_3_sum',
 'type_4_sum',
 'property_condition_score',
 'property_condition_tier',
 'ownname1',
 'ownname2',
 'namekey',
 'ownadd1',
 'ownadd2',
 'owncity',
 'ownstate',
 'ownerzip',
 'ownzip2',
 'Business Name',
 'License Id',
 'address',
 'Customer Id',
 'Dist/Account No',
 'RENTAL',
 'CAN_GROUP',
 'CAN_OWNCLASS',
 'GEOLATLON',
 'GEOHASH',
 'fee',
 'std_fee']

THERE ARE PROBLEMS IN THE RENTAL DATA SET THAT DROPS RECORDS HERE

In [ ]:
# violations['vtypes'] = violations.apply(lambda x: x['Violation(s)'].split('\n'),axis=1)
# v_flat = violations.explode('vtypes')
vsummary = v_flat.groupby('APN')['Violation(s)'].value_counts().reset_index(name='vcount')
vs2 = vsummary.groupby(['APN'])['vcount'].sum().reset_index(name='violations_total')
vs2


,APN,violations_total
0,0,1
1,1007104014,1
2,1007104057,3
3,1007104065,6
4,1007104138,2
...,...,...
1426,1007229348,1
1427,1007230664,2
1428,1007230680,1
1429,1007232500,1


In [ ]:
#rentals.query('acctid == "1007229100"')

In [ ]:

import re
!cp "drive/My Drive/pita 2021/text_cluster.py" .
from text_cluster import assign_clusters
from text_cluster import text_similarity
from text_cluster import cluster_strings

def format_contact(s):
  print(s)
  cinfo = re.sub(r'[,.;:-]','', s).split("\n")
  name = cinfo[0]
  address = ""
  print(cinfo)
  if "Property Owner" in cinfo[0]:
    if len(cinfo) > 2:
      name = cinfo[1].replace("'","").upper()
      address = re.sub(r',* *[A-Z][A-Z] \d\d\d\d\d\'*$',"",cinfo[2]).replace("'","").upper()
    else:
      match = re.search(r' \d+ ',cinfo[1])
      if match:
        tokens = cinfo[1].upper().split(" ")
        numbers = [i for i,token in enumerate(tokens) if re.search(r'\d',token)]
        if len(numbers) > 0:
          name = " ".join(tokens[0:numbers[0]])
          address = " ".join(tokens[numbers[0]:-1])
  else:
    print("BAD RECORD ",s)
  return pd.Series([name, address], index=['Contact_Name','Contact_Address'])


violations[['Contact_Name','Contact_Address']] = violations['Contacts'].apply(format_contact)
violations['address'] = violations.fillna("").apply(lambda x: " ".join([str(x['Street Number']),str(x['Street Direction']),str(x['Street Name']),str(x['Street Type'])]).upper(),axis=1)
violations.APN = violations.apply(lambda x: x.APN.strip(u'\u200b'),axis=1)
violations.address = violations.apply(lambda x: re.sub(r' +',' ',x.address).strip(),axis=1)

Property Owner MANOKEY CALVIN L507 DUNNS Ct CAMBRIDGE, MD 21613Home phone: Work phone: Cell phone:
['Property Owner MANOKEY CALVIN L507 DUNNS Ct CAMBRIDGE MD 21613Home phone Work phone Cell phone']


IndexError: ignored

####Cluster the violations
Assign clusters by contact name and contact address, split from the Contact column.

In [ ]:
%%time
myquery = violations.query('(Open_Date > "2018-12-31")',engine='python')
names = cluster_strings(myquery.Contact_Name.array,'c_contact',metric='jaccard',threshold=0.1)
addresses = cluster_strings(myquery.Contact_Address.array,'c_address',metric='jaccard',threshold=0.1)

AttributeError: ignored

In [ ]:
violations['contact_cluster'] = violations.apply(lambda x: names.get(x.Contact_Name,-1), axis=1)
violations['contactadd_cluster'] = violations.apply(lambda x: addresses.get(x.Contact_Address,-1), axis=1)

NameError: ignored

#### At this point we have clusters for all the code violations

Some tax accounts are associated with more than one cluster.  This can be due to entry errors in the complaint data, or changes in ownership over the complaint period. The default behavior, used as the fixed behavior here, is to return the least ambiguous dominant cluster id.  What this means practically is if the most often cited cluster == -1 (no cluster), but the second most frequent citation in the period is a valid cluster, that valid cluster is assinged to the tax id.  If more than one valid cluster is cited, the most frequently cited is used.

Some of the violations have no tax acctid, in some cases because of address spelling issues.  Some of these can be backfilled.  The ones we can fix, we do.  Many of the others are just vacant lots which will drop out on the join with properties licenesed for rental.

In [ ]:
def best_cluster(cnumbers):
  if len(cnumbers) > 1 and cnumbers.index[0] < 0:
    return cnumbers.index[1]
  else:
    return cnumbers.index[0]
    
vc_df = pd.DataFrame(violations.groupby(['APN'])['contactadd_cluster'].agg(lambda x:best_cluster(x.value_counts())))#.to_dict()
cadd_c = violations.groupby(['APN'])['contact_cluster'].agg(lambda x:best_cluster(x.value_counts()))
vc_df = vc_df.merge(cadd_c,left_index=True,right_index=True)

# fix tax account id's where possible
vc_df = vc_df.merge(vs2, on='APN',how='inner')
vc_df

### Save result as violation_clusters.csv


In [ ]:
vc_df.to_csv('drive/My Drive/pita 2020/violation_clusters.csv')

In [ ]:
vsample = violations.query('(Open_Date > "2018-12-31")')[['APN','address','contact_cluster','Contact_Name','contactadd_cluster','Contact_Address']]
vsample.APN = vsample.APN.astype(str)
vsample = vsample.rename(columns={'APN':'acctid'})
vsample.acctid = vsample.apply(lambda x: x.acctid.strip(u'\u200b'),axis=1)
vsample.address = vsample.apply(lambda x: re.sub(r' +',' ',x.address).strip(),axis=1)

noacctid = vsample.query('acctid == ""',engine='python').address.array
found_acctids = {v[0]:v[1] for v in df.query('address in @noacctid')[['address','acctid']].values}
vsample.acctid = vsample.apply(lambda x: found_acctids.get(x.address,x.acctid),axis=1)
vsample.query('acctid == ""',engine='python').address.array
#.query('Contact_Name.str.contains("GLG")',engine='python')